In [ ]:
import pandas as pd
import numpy as np

In [5]:
file_name = 'preprocessed_data/kneighbors_05_06_2025'

train_data_file_path = f"../{file_name}_train.csv"
test_data_file_path = f"../{file_name}_test.csv"

train = pd.read_csv(train_data_file_path)
test = pd.read_csv(test_data_file_path)

C:\Users\irela\AppData\Local\Temp\ipykernel_20020\2062783055.py:6: DtypeWarning: Columns (2,16) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv(train_data_file_path)


In [4]:
train.head()

,DerateInNextTwentyFourHours,spn,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,FuelLtd,...,fmi_23,fmi_29,fmi_3,fmi_31,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9
0,0,829,0.197467,-0.629948,0.416268,-0.551157,-0.701429,0.057964,-0.976154,-0.524722,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,96,0.197467,-0.629948,0.416268,-0.551157,-0.701429,0.057964,-0.976154,-0.524722,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,96,-8.917271,0.206478,0.481970,0.148371,0.173607,-1.141057,0.268311,0.323081,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,829,-8.917271,0.206478,0.481970,0.148371,0.173607,-1.141057,0.268311,0.323081,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,792,-0.069899,0.406607,0.525771,0.279124,-0.083757,0.054851,0.690035,1.057251,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [16]:
def add_time_based_features_to_feature_col(df, feature_col, prev_num_hours=24):
    # Add time-based features to a specific feature column in the DataFrame
    print(f"Adding time-based features to {feature_col} with a rolling window of {prev_num_hours} hours")
    df = df.copy()
    df['EventTimeStamp'] = pd.to_datetime(df['EventTimeStamp'])

    grouped_results = []

    # Loop through each truck to keep calculations separate
    for equipment_id, group in df.groupby('EquipmentID'):
        #print(f"Processing EquipmentID: {equipment_id}")
        group = group.sort_values('EventTimeStamp').set_index('EventTimeStamp')

        # Create time-based features
        window = f'{prev_num_hours}h'
        group[f'Mean{feature_col}Last{prev_num_hours}hr'] = group[feature_col].rolling(window).mean()
        group[f'Max{feature_col}Last{prev_num_hours}hr'] = group[feature_col].rolling(window).max()
        group[f'Min{feature_col}Last{prev_num_hours}hr'] = group[feature_col].rolling(window).min()
        group[f'Std{feature_col}Last{prev_num_hours}hr'] = group[feature_col].rolling(window).std()
        group[f'Sum{feature_col}Last{prev_num_hours}hr'] = group[feature_col].rolling(window).sum()

        group['EquipmentID'] = equipment_id
        grouped_results.append(group.reset_index())

    return pd.concat(grouped_results, ignore_index=True)


In [17]:
testing_train_data = add_time_based_features_to_feature_col(train, 'BarometricPressure', prev_num_hours=24)
testing_train_data.head()

Adding time-based features to BarometricPressure with a rolling window of 24 hours


,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9,MeanBarometricPressureLast24hr,MaxBarometricPressureLast24hr,MinBarometricPressureLast24hr,StdBarometricPressureLast24hr,SumBarometricPressureLast24hr
0,2015-05-04 15:38:35,50353,1327,0.075937,1.073158,0.416268,1.900459,0.019188,0.477227,0.918218,...,0.0,0.0,0.0,0.0,0.0,0.075937,0.075937,0.075937,NaN,0.075937
1,2015-05-04 15:42:02,50353,1327,0.039478,0.992382,0.580522,1.442824,-0.052873,0.768515,0.756643,...,0.0,0.0,0.0,0.0,0.0,0.057708,0.075937,0.039478,0.025780,0.115415
2,2015-05-07 06:52:14,596,1327,0.501292,1.079067,0.471019,-0.583845,-0.701429,-0.348484,-0.849904,...,0.0,0.0,0.0,0.0,0.0,0.501292,0.501292,0.501292,NaN,0.501292
3,2015-05-07 10:25:59,596,1327,0.282538,-0.033368,0.492920,-0.440017,0.533916,0.320324,0.901364,...,0.0,0.0,0.0,0.0,0.0,0.391915,0.501292,0.282538,0.154682,0.783829
4,2015-05-19 12:02:55,3216,1327,0.136702,1.091369,0.471019,-1.041480,-0.135230,0.269427,0.231617,...,0.0,0.0,0.0,0.0,1.0,0.136702,0.136702,0.136702,NaN,0.136702


In [30]:
def add_agg_features_to_cols(df, prev_num_hours=24, filter_out_fmi=True):
    # Calculate aggregate features for all numeric columns
    if filter_out_fmi:
        cols_to_exclude = ['spn', 'EventTimeStamp', 'EquipmentID', 'NextDerateTime', 'HoursUntilNextDerate', 'FullDerate', 'DerateInNextTwoHours', 'DerateInNextTwentyFourHours']
        for i in range(0, 32):
            cols_to_exclude.append(f'fmi_{i}')
    else:
        cols_to_exclude = ['spn', 'EventTimeStamp', 'EquipmentID', 'NextDerateTime', 'HoursUntilNextDerate', 'FullDerate', 'DerateInNextTwoHours', 'DerateInNextTwentyFourHours']
    
    cols_to_include = df.select_dtypes(include=[np.number]).columns.difference(cols_to_exclude).tolist()
    for col in cols_to_include:
        df = add_time_based_features_to_feature_col(df, col, prev_num_hours)

    return df

In [31]:
train_aggregated = add_agg_features_to_cols(train, prev_num_hours=24)
train_aggregated.head()

Adding time-based features to BarometricPressure with a rolling window of 24 hours
Adding time-based features to DistanceLtd with a rolling window of 24 hours
Adding time-based features to EngineCoolantTemperature with a rolling window of 24 hours
Adding time-based features to EngineLoad with a rolling window of 24 hours
Adding time-based features to EngineOilPressure with a rolling window of 24 hours
Adding time-based features to EngineOilTemperature with a rolling window of 24 hours
Adding time-based features to EngineRpm with a rolling window of 24 hours
Adding time-based features to FuelLtd with a rolling window of 24 hours
Adding time-based features to FuelRate with a rolling window of 24 hours
Adding time-based features to IntakeManifoldTemperature with a rolling window of 24 hours
Adding time-based features to LampStatus with a rolling window of 24 hours


,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,MeanIntakeManifoldTemperatureLast24hr,MaxIntakeManifoldTemperatureLast24hr,MinIntakeManifoldTemperatureLast24hr,StdIntakeManifoldTemperatureLast24hr,SumIntakeManifoldTemperatureLast24hr,MeanLampStatusLast24hr,MaxLampStatusLast24hr,MinLampStatusLast24hr,StdLampStatusLast24hr,SumLampStatusLast24hr
0,2015-05-04 15:38:35,50353,1327,0.075937,1.073158,0.416268,1.900459,0.019188,0.477227,0.918218,...,0.245640,0.245640,0.245640,NaN,0.245640,-0.429428,-0.429428,-0.429428,NaN,-0.429428
1,2015-05-04 15:42:02,50353,1327,0.039478,0.992382,0.580522,1.442824,-0.052873,0.768515,0.756643,...,-0.107209,0.245640,-0.460057,0.499003,-0.214417,-0.414385,-0.399341,-0.429428,0.021275,-0.828770
2,2015-05-07 06:52:14,596,1327,0.501292,1.079067,0.471019,-0.583845,-0.701429,-0.348484,-0.849904,...,-0.177778,-0.177778,-0.177778,NaN,-0.177778,-0.399341,-0.399341,-0.399341,NaN,-0.399341
3,2015-05-07 10:25:59,596,1327,0.282538,-0.033368,0.492920,-0.440017,0.533916,0.320324,0.901364,...,-0.064867,0.048045,-0.177778,0.159681,-0.129734,-0.338454,-0.277567,-0.399341,0.086108,-0.676908
4,2015-05-19 12:02:55,3216,1327,0.136702,1.091369,0.471019,-1.041480,-0.135230,0.269427,0.231617,...,0.104500,0.104500,0.104500,NaN,0.104500,1.640382,1.640382,1.640382,NaN,1.640382


In [32]:
test_aggregated = add_agg_features_to_cols(test, prev_num_hours=24)
test_aggregated.head()

Adding time-based features to BarometricPressure with a rolling window of 24 hours
Adding time-based features to DistanceLtd with a rolling window of 24 hours
Adding time-based features to EngineCoolantTemperature with a rolling window of 24 hours
Adding time-based features to EngineLoad with a rolling window of 24 hours
Adding time-based features to EngineOilPressure with a rolling window of 24 hours
Adding time-based features to EngineOilTemperature with a rolling window of 24 hours
Adding time-based features to EngineRpm with a rolling window of 24 hours
Adding time-based features to FuelLtd with a rolling window of 24 hours
Adding time-based features to FuelRate with a rolling window of 24 hours
Adding time-based features to IntakeManifoldTemperature with a rolling window of 24 hours
Adding time-based features to LampStatus with a rolling window of 24 hours


,EventTimeStamp,spn,EquipmentID,BarometricPressure,DistanceLtd,EngineCoolantTemperature,EngineLoad,EngineOilPressure,EngineOilTemperature,EngineRpm,...,MeanIntakeManifoldTemperatureLast24hr,MaxIntakeManifoldTemperatureLast24hr,MinIntakeManifoldTemperatureLast24hr,StdIntakeManifoldTemperatureLast24hr,SumIntakeManifoldTemperatureLast24hr,MeanLampStatusLast24hr,MaxLampStatusLast24hr,MinLampStatusLast24hr,StdLampStatusLast24hr,SumLampStatusLast24hr
0,2019-01-11 12:51:18,96,301,0.258232,0.176919,-0.733512,0.462178,2.541351,-0.148922,0.585179,...,-0.883475,-0.883475,-0.883475,NaN,-0.883475,-0.277567,-0.277567,-0.277567,NaN,-0.277567
1,2019-01-11 12:51:18,829,301,0.258232,0.176919,-0.733512,0.462178,2.541351,-0.148922,0.585179,...,-0.883475,-0.883475,-0.883475,0.000000,-1.766949,-0.277567,-0.277567,-0.277567,0.000000,-0.555134
2,2019-01-11 13:11:40,96,301,-8.917271,0.206478,0.481970,0.148371,0.173607,-1.141057,0.268311,...,-0.370669,0.654944,-0.883475,0.888206,-1.112006,-0.318158,-0.277567,-0.399341,0.070307,-0.954475
3,2019-01-11 13:11:40,829,301,-8.917271,0.206478,0.481970,0.148371,0.173607,-1.141057,0.268311,...,-0.114266,0.654944,-0.883475,0.888206,-0.457062,-0.338454,-0.277567,-0.399341,0.070307,-1.353817
4,2019-01-11 13:13:38,96,301,0.258232,0.176961,-0.405004,-0.616533,-0.032284,-0.260603,-0.841508,...,-0.084626,0.654944,-0.883475,0.772059,-0.423132,-0.326277,-0.277567,-0.399341,0.066699,-1.631383


In [33]:
target_columns = ['HoursUntilNextDerate', 'DerateInNextTwoHours', 'DerateInNextTwentyFourHours']
y_train = train_aggregated[target_columns].copy()
y_test = test_aggregated[target_columns].copy()

train_filtered = train_aggregated.drop(columns=['EventTimeStamp', 'EquipmentID', 'NextDerateTime', 'HoursUntilNextDerate', 'FullDerate', 'DerateInNextTwoHours', 'DerateInNextTwentyFourHours'])
test_filtered = test_aggregated.drop(columns=['EventTimeStamp', 'EquipmentID', 'NextDerateTime', 'HoursUntilNextDerate', 'FullDerate', 'DerateInNextTwoHours', 'DerateInNextTwentyFourHours'])

for target in target_columns:
    print(f"Processing target: {target}")
    new_train_df = pd.concat([y_train[target], train_filtered], axis=1)
    new_test_df = pd.concat([y_test[target], test_filtered], axis=1)
    new_train_df.to_csv(f'../preprocessed_data/{target}_kneighbors_aggfuncs_slim_05_08_train.csv', index=False)
    new_test_df.to_csv(f'../preprocessed_data/{target}_kneighbors_aggfuncs_slim_05_08_test.csv', index=False)
    new_train_df = pd.DataFrame()
    new_test_df = pd.DataFrame()
    print(f"Saved {target} dataframes for training and testing.")

Processing target: HoursUntilNextDerate
Saved HoursUntilNextDerate dataframes for training and testing.
Processing target: DerateInNextTwoHours
Saved DerateInNextTwoHours dataframes for training and testing.
Processing target: DerateInNextTwentyFourHours
Saved DerateInNextTwentyFourHours dataframes for training and testing.
